In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_27c_test2.pth

In [1]:
!nvidia-smi --query-gpu=name --format=csv,noheader | head -n 1

NVIDIA GeForce RTX 4090


In [ ]:
import io
import time
import torch
import numpy as np
import PIL
import torchaudio
import matplotlib.pyplot as plt
from einops import rearrange
from IPython.display import Audio
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc
from spauq.core.metrics import spauq_eval
import cdpam
class Config: pass

In [ ]:
checkpoint = torch.load("Stereo_Li_27c_test2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

In [ ]:
cdpam_loss = cdpam.CDPAM()

In [ ]:
MUSDB = load_dataset("danjacobellis/musdb_segments", split='validation')

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
def walloc_compress(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio_mix']['bytes'],normalize=False)
        x = x.to(torch.float)
        x = x - x.mean()
        max_abs = x.abs().max()
        x = x / (max_abs + 1e-8)
        x = x/2
        
        L = x.shape[-1]
        x_padded = pad(x.unsqueeze(0), 2**16).to(device)
        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        ℓ = Y.shape[-1]
        Y = pad(Y,256)
        Y = rearrange(Y, 'b c (w h) -> b c w h', h=256).to("cpu")
        webp = walloc.latent_to_pil(Y,codec.latent_bits,3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
    
        t0 = time.time()
        Y = walloc.pil_to_latent([PIL.Image.open(buff)], codec.latent_dim, codec.latent_bits, 3).to(device)
        X_hat = codec.decoder(rearrange(Y.to(device), 'b c h w -> b c (h w)')[:,:,:ℓ])
        x_hat = codec.wavelet_synthesis(X_hat,codec.J)
        x_hat = codec.post(x_hat)
        x_hat = codec.clamp(x_hat[0,:,:L])
        decode_time = time.time() - t0
    
        bps = 8*len(webp_bytes)/(x.numel())
        PSNR = - 10*np.log10(torch.nn.functional.mse_loss(x,x_hat.to("cpu")))
        SDR = spauq_eval(x,x_hat.to("cpu"),fs=fs)
        SSDR = SDR['SSR']
        SRDR = SDR['SRR']
        cdpam = cdpam_loss.forward(x.to(device),x_hat).mean().item()
        
    return {
        'compressed': webp_bytes,
        'encode_time': encode_time,
        'decode_time': decode_time,
        'bps': bps,
        'L': L,
        'PSNR': PSNR,
        'SSDR': SSDR,
        'SRDR': SRDR,
        'CDPAM': cdpam
    }

In [ ]:
device = "cuda"
codec = codec.to(device)
gpu = MUSDB.map(
    walloc_compress,
    writer_batch_size=16,
)
gpu = gpu.cast_column('compressed',Image())

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [9]:
def walloc_compress_cpu(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio_mix']['bytes'],normalize=False)
        x = x.to(torch.float)
        x = x - x.mean()
        max_abs = x.abs().max()
        x = x / (max_abs + 1e-8)
        x = x/2
        
        L = x.shape[-1]
        x_padded = pad(x.unsqueeze(0), 2**16).to(device)
        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        ℓ = Y.shape[-1]
        Y = pad(Y,256)
        Y = rearrange(Y, 'b c (w h) -> b c w h', h=256).to("cpu")
        webp = walloc.latent_to_pil(Y,codec.latent_bits,3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
    
        t0 = time.time()
        Y = walloc.pil_to_latent([PIL.Image.open(buff)], codec.latent_dim, codec.latent_bits, 3).to(device)
        X_hat = codec.decoder(rearrange(Y.to(device), 'b c h w -> b c (h w)')[:,:,:ℓ])
        x_hat = codec.wavelet_synthesis(X_hat,codec.J)
        x_hat = codec.post(x_hat)
        x_hat = codec.clamp(x_hat[0,:,:L])
        decode_time = time.time() - t0
        
    return {
        'cpu_encode_time': encode_time,
        'cpu_decode_time': decode_time,
    }

In [11]:
device = "cpu"
codec = codec.to(device)
cpu = MUSDB.map(walloc_compress_cpu, writer_batch_size=16)
combined = gpu.add_column('cpu_encode_time',cpu['cpu_encode_time'])
combined = combined.add_column('cpu_decode_time',cpu['cpu_decode_time'])

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [12]:
metrics = [
    'encode_time',
    'decode_time',
    'bps',
    'PSNR',
    'SSDR',
    'SRDR',
    'CDPAM',
    'cpu_encode_time',
    'cpu_decode_time',
]

In [13]:
for metric in metrics:
    μ = np.mean(combined[metric])
    print(f"{metric}: {μ}")

encode_time: 0.011810283624488889
decode_time: 0.0044139505342672795
bps: 0.2097470960544266
PSNR: 33.31613411794182
SSDR: 22.522951690375507
SRDR: 8.06101032870686
CDPAM: 0.00022564448386740442
cpu_encode_time: 0.035690814484166736
cpu_decode_time: 0.34584116753731065


In [14]:
combined.push_to_hub("danjacobellis/MUSDB_Stereo_Li_27c_test2",split='validation')

Uploading the dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/MUSDB_Stereo_Li_27c_test2/commit/d4db3c20e13f0c4b7c8a6b617f6b8b12078047ba', commit_message='Upload dataset', commit_description='', oid='d4db3c20e13f0c4b7c8a6b617f6b8b12078047ba', pr_url=None, pr_revision=None, pr_num=None)

---

In [11]:
# L = 1048064
# sample = MUSDB[5]
# x, fs = torchaudio.load(sample['audio']['bytes'])
# x = x[:,:L]
# codec = codec.to("cuda")
# x = x.to("cuda").unsqueeze(0)
# with torch.no_grad():
#     X = codec.wavelet_analysis(x,codec.J)
#     Y = codec.encoder(X)
#     Z = codec.decoder(Y)
#     z = codec.wavelet_synthesis(Z,codec.J)
#     z = codec.post(z)
# x = x.to("cpu")[0]
# z = z.to("cpu")[0]

In [12]:
# Audio(x.numpy(),rate=44100)

In [13]:
# Audio(z.numpy(),rate=44100)

In [14]:
# start, end = 56500, 57000
# plt.figure(figsize=(8, 4), dpi=180)
# plt.plot(x[0, start:end], alpha=0.5, c='b', label='Ch.1 (Uncompressed)')
# plt.plot(z[0, start:end], alpha=0.5, c='g', label='Ch.1 (WaLLoC)')
# plt.plot(x[1, start:end], alpha=0.5, c='r', label='Ch.2 (Uncompressed)')
# plt.plot(z[1, start:end], alpha=0.5, c='purple', label='Ch.2 (WaLLoC)')

# plt.ylim([-1,0.6])
# plt.legend(loc='lower center')
# plt.box(False)
# plt.xticks([])
# plt.yticks([])
# # plt.savefig("test.svg")